In [1]:
import mlflow
import shutil
import time
import torch

from functools import partial
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from tqdm import tqdm

from nba_betting_ai.consts import proj_paths
from nba_betting_ai.data.processing import add_prefix
from nba_betting_ai.model.bayesian import BayesianResultPredictor
from nba_betting_ai.model.inputs import prepare_scalers
from nba_betting_ai.training.dataset import NBADataset
from nba_betting_ai.training.pipeline import prepare_data

In [2]:
team_stats_season = [
    'season_games',
    'season_wins', 'season_pts_for', 'season_pts_against',
    'season_wins_avg', 'season_pts_for_avg', 'season_pts_against_avg',
]
team_stats_last_5 = [
    'last_5_wins',
    'last_5_pts_for_avg', 'last_5_pts_against_avg',
    'last_5_pts_for_total', 'last_5_pts_against_total',
]
target_general = ['away_win', 'score_final_diff']
target_team = 'score_final'
identification_cols = [
    'game_id', 'home_team_abbreviation', 'away_team_abbreviation',
]
other_features = [
    'home_score', 'away_score', 'time_remaining', 'score_diff'
]

prefix_home = partial(add_prefix, prefix='home', return_type='list')
prefix_away = partial(add_prefix, prefix='away', return_type='list')
all_features = identification_cols \
            + target_general \
            + prefix_home([target_team]) + prefix_away([target_team]) \
            + prefix_home(team_stats_season) + prefix_away(team_stats_season) \
            + prefix_home(team_stats_last_5) + prefix_away(team_stats_last_5)


In [3]:
data_params = {
    'seasons': 2,
    'seed': 66,
    'test_size': 0.2,
    'n': 20,
    'frac': None
}

team_features = [
    'away_season_wins_avg', 'home_season_wins_avg',
    'away_season_pts_diff_avg', 'away_last_5_pts_diff_avg',
    'home_season_pts_diff_avg', 'home_last_5_pts_diff_avg'
]
target_general = ['final_score_diff']

params = {
    'learning_rate': 0.05,
    'lr_gamma': 0.85,
    'batch_size': 128,
    'epochs': 300,
    'eval_freq': 10
}

model_config = {
    'embedding_dim': 4, #8,
    'team_hidden_dim': 16, # 32,
    'team_layers': 3,
    'res_hidden_dim': 16, #32,
    'res_layers': 4,
    'time_scaling': True
}

config = {
    'features': team_features,
    'target': target_general,
    'training': {
        'params': params,
        'model_config': model_config
    }
}

In [4]:
from collections.abc import MutableMapping
from omegaconf import OmegaConf
 
def convert_flatten(d, parent_key ='', sep ='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
 
        if isinstance(v, MutableMapping):
            items.extend(convert_flatten(v, new_key, sep = sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [6]:
now = time.strftime("%Y%m%d-%H%M%S")
config = OmegaConf.load(proj_paths.config.default)
config_archive = proj_paths.config.default.with_stem(f'config-{now}')

scalers_path = proj_paths.output / 'scalers.pkl'

data_prepared = prepare_data(**config['data_params'])
scalers = prepare_scalers(data_prepared.X_train, team_features, scalers_path)

In [14]:
now = time.strftime("%Y%m%d-%H%M%S")
config = OmegaConf.load(proj_paths.config.default)
config_archive = proj_paths.config.default.with_stem(f'config-{now}')

scalers_path = proj_paths.output / 'scalers.pkl'

data_prepared = prepare_data(**config['data_params'])
scalers = prepare_scalers(data_prepared.X_train, team_features, scalers_path)

train_dataset = NBADataset(team_features, target_general, data_prepared.X_train, data_prepared.teams, scalers)
test_dataset = NBADataset(team_features, target_general, data_prepared.X_test, data_prepared.teams, scalers)

train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BayesianResultPredictor(
    team_count=len(data_prepared.teams),
    team_features=len(team_features)//2,
    **model_config
).to(device)
learning_rate = params['learning_rate']
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = params.get('lr_gamma', 1.0))
const = 0.5 * torch.log(2 * torch.tensor(torch.pi, dtype=torch.float64)).to(device)

create_experiment = mlflow.get_experiment_by_name(config['experiment_name']) is None
if create_experiment:
    mlflow.create_experiment(
        name=config['experiment_name'],
        tags={
            'author': 'Tom Bukic',
            'company': 'accelerate.ai'
        }
    )
    create_experiment = False

mlflow.set_experiment(config['experiment_name'])

run_name = f'bayesian_model-{now}'
best_loss = float('inf')
with mlflow.start_run(run_name=run_name):
    flat_config = convert_flatten(config)
    mlflow.log_params(flat_config)
    mlflow.log_artifact(scalers_path)
    mlflow.log_artifact(proj_paths.config.default)
    shutil.copy(proj_paths.config.default.as_posix(), config_archive.as_posix())
    last_update = 0
    with tqdm(total=params['epochs']*len(train_loader), desc='Training Progress') as progress_bar:
        for epoch in range(params['epochs']):
            model.train()
            for i, data in enumerate(train_loader):
                step = epoch * params['batch_size'] + i
                data = {
                    k: v.to(device)
                    for k, v in data.items()
                }
                target = data.pop('y')
                optimizer.zero_grad()
                output = model(**data)
                mu, logvar = torch.chunk(output, 2, dim=-1)
                logvar = torch.clamp(logvar, min=-10, max=10)
                loss = const + logvar + (target - mu)**2 / torch.exp(logvar)
                loss = torch.mean(loss)
                loss.backward()
                optimizer.step()
                if step % 20 == 0:
                    progress_bar.set_description(f'loss: {loss:.4f}')
                    progress_bar.update(20)
                    mlflow.log_metric('train_loss', loss.item(), step=step)
                    last_update = step
            scheduler.step(epoch=epoch)
            if last_update < step:
                progress_bar.set_description(f'loss: {loss:.4f}')
                progress_bar.update(step - last_update)
                last_update = step
            mlflow.log_metric('train_loss', loss.item(), step=step)
            if epoch % params['eval_freq'] != 0 and epoch < params['epochs'] - 1:
                continue
            model.eval()
            correct = 0
            total = 0
            total_loss = 0
            avg_var = 0
            n = 0
            with torch.no_grad():
                for i, data in enumerate(test_loader):
                    data = {
                        k: v.to(device)
                        for k, v in data.items()
                    }
                    target = data.pop('y')
                    output = model(**data)
                    mu, logvar = torch.chunk(output, 2, dim=-1)
                    logvar = torch.clamp(logvar, min=-10, max=10)
                    var = torch.exp(logvar)
                    loss = const + logvar + (target - mu)**2 / var
                    total_loss += torch.mean(loss)
                    next_n = n + len(target)
                    avg_var = avg_var*(n/next_n) + torch.mean(var).item()*(len(target)/next_n)
                    n = next_n
                    
                    total += target.size(0)
                    correct += (mu*target >= 0).sum().item()
            avg_loss = total_loss.item() / len(test_loader)
            accuracy = correct / n
            model_path = proj_paths.output / f'bayesian_model-{now}-loss_{str(avg_loss).replace('.', '_')}.pth'
            torch.save(model.state_dict(), model_path)
            mlflow.log_artifact(model_path)
            if avg_loss < best_loss:
                best_loss = avg_loss
                best_model_path = proj_paths.output / 'bayesian_model_best.pth'
                torch.save(model.state_dict(), best_model_path)
                mlflow.log_artifact(best_model_path)
            mlflow.log_metric('eval_accuracy', accuracy, step=step)
            mlflow.log_metric('eval_var', avg_var, step=step)
            mlflow.log_metric('eval_loss', avg_loss, step=step)

2025/01/27 20:43:19 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2025/01/27 20:43:19 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
loss: 1.5094:   0%|          | 60/76200 [00:02<44:30, 28.52it/s]
2025/01/27 20:43:22 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/01/27 20:43:22 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run bayesian_model-20250127-204316 at: http://mlflow:5000/#/experiments/664344747479239756/runs/ef2460b794374aa283a6c0c4cb85a430
🧪 View experiment at: http://mlflow:5000/#/experiments/664344747479239756


KeyboardInterrupt: 